# YOLOv8 benchmark, export and serve in AML

## Benchmarking

In [ ]:
# Install YOLOv8
%pip install ultralytics
%pip install onnx

In [2]:
# Import YOLOv8 benchmarks
from ultralytics.yolo.utils.benchmarks import benchmark

In [ ]:
# Benchmark model
benchmark(model='yolov8l.pt', imgsz=640, half=False, device=0)   # device 0 for GPU or "cpu" for CPU)
benchmark(model='yolov8n.pt', imgsz=640, half=False, device=0)   # device 0 for GPU or "cpu" for CPU)

### Results
| VM Type | Time nano (ms)| Time large (ms) | VM price per hour (USD) | Price per 1M images nano (USD) | Price per 1M images large (USD) |
| ------- | ----------- | ------------ | ----------- | ----------- | ----------- |
| STANDARD_D2AS_V4 | 123 | 1782 | 0.115 | 3.39 | 56.93 |
| STANDARD_NC24ADS_A100_V4 | 5 | 8 | 4.78 | 6.64 | 10.62 |
| STANDARD_D96A_V4 | 36 | 139 | 4.9 |  49 | 189.18 |

In [20]:
def price_per_1M_images(speed, price):
    return round(price / (1000*60*60/speed)*1000000, 2)

print("----- nano ------")
print(f"STANDARD_D2AS_V4 nano: {price_per_1M_images(123, 0.115)} USD")
print(f"STANDARD_D96A_V4 nano: {price_per_1M_images(36, 4.9)} USD")
print(f"STANDARD_NC24ADS_A100_V4 nano: {price_per_1M_images(5, 4.78)} USD")
print()
print("----- large ------")
print(f"STANDARD_D2AS_V4 large: {price_per_1M_images(1782, 0.115)} USD")
print(f"STANDARD_D96A_V4 large: {price_per_1M_images(139, 4.9)} USD")
print(f"STANDARD_NC24ADS_A100_V4 large: {price_per_1M_images(8, 4.78)} USD")

----- nano ------
STANDARD_D2AS_V4 nano: 3.93 USD
STANDARD_D96A_V4 nano: 49.0 USD
STANDARD_NC24ADS_A100_V4 nano: 6.64 USD

----- large ------
STANDARD_D2AS_V4 large: 56.93 USD
STANDARD_D96A_V4 large: 189.19 USD
STANDARD_NC24ADS_A100_V4 large: 10.62 USD


## Export to ONNX

In [ ]:
from ultralytics import YOLO
import os

# Load models
model_yolov8n = YOLO('models/yolov8n/1/yolov8n.pt')  # load an official model
model_yolov8l = YOLO('models/yolov8l/1/yolov8l.pt')  # load an official model


In [ ]:

# Export models
model_yolov8n.export(format='onnx')
model_yolov8l.export(format='onnx')

# Rename models
os.replace('models/yolov8n/1/yolov8n.onnx', 'models/yolov8n/1/model.onnx')
os.replace('models/yolov8l/1/yolov8l.onnx', 'models/yolov8l/1/model.onnx')

# Delete pytorch version of model
os.remove('models/yolov8n/1/yolov8n.pt')
os.remove('models/yolov8l/1/yolov8l.pt')

## Serve in Azure Machine Learning

In [ ]:
%pip install numpy --user --upgrade
%pip install tritonclient[http] --user --upgrade
%pip install pillow --user --upgrade
%pip install gevent --user --upgrade
%pip install azure-ai-ml --user --upgrade

In [4]:
# Variables
subscription_id = "d3b7888f-c26e-4961-a976-ff9d5b31dfd3"
resource_group = "toml"
workspace_name = "tomlsw"
endpoint_name = "tomaskubica1423"

In [5]:
# Login
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id,
    resource_group,
    workspace_name,
)

In [6]:
# Create endpoint and deployment
from azure.ai.ml.entities import ManagedOnlineEndpoint

# Create endpoint
endpoint = ManagedOnlineEndpoint(name=endpoint_name, auth_mode="key")
endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

In [7]:
# Create deployment
from azure.ai.ml.entities import ManagedOnlineDeployment, Model

deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=Model(path="./models", type="triton_model"),
    instance_type="Standard_D2as_v4",
    instance_count=1,
)

ml_client.online_deployments.begin_create_or_update(deployment).result()

# Update traffic
endpoint.traffic = {"blue": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

Check: endpoint tomaskubica1423 exists
Your file exceeds 100 MB. If you experience low speeds, latency, or broken connections, we recommend using the AzCopyv10 tool for this file transfer.

Example: azcopy copy 'C:\Users\tokubica\OneDrive - Microsoft\git\ai-demos\yolo\models' 'https://tomlsw6681050375.blob.core.windows.net/azureml-blobstore-c514ab43-63ac-4d39-abb7-0dba62c004bb/LocalUpload/791de9f58a77202d3ec1040dbde83bbb/models' 

See https://docs.microsoft.com/azure/storage/common/storage-use-azcopy-v10 for more information.
Uploading models (187.74 MBs): 100%|##########| 187739575/187739575 [00:08<00:00, 23120501.19it/s]




........................................................................................................

Readonly attribute principal_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>
Readonly attribute tenant_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>


ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://tomaskubica1423.swedencentral.inference.ml.azure.com/', 'openapi_uri': 'https://tomaskubica1423.swedencentral.inference.ml.azure.com/swagger.json', 'name': 'tomaskubica1423', 'description': None, 'tags': {}, 'properties': {'azureml.onlineendpointid': '/subscriptions/d3b7888f-c26e-4961-a976-ff9d5b31dfd3/resourcegroups/toml/providers/microsoft.machinelearningservices/workspaces/tomlsw/onlineendpoints/tomaskubica1423', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/d3b7888f-c26e-4961-a976-ff9d5b31dfd3/providers/Microsoft.MachineLearningServices/locations/swedencentral/mfeOperationsStatus/oe:c514ab43-63ac-4d39-abb7-0dba62c004bb:8e542b50-6377-45af-a4b9-ee53d0f86cf4?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/d3b7888f-c26e-4961-a976-ff9d5b31dfd3/resourceGroups/toml/providers/Microsoft.MachineLearningServices/workspaces

In [ ]:
# Debug - run triton server locally
# !docker pull nvcr.io/nvidia/tritonserver:23.05-py3
# !docker run -it --rm -p8000:8000 -p8001:8001 -p8002:8002 -v ./models:/models nvcr.io/nvidia/tritonserver:23.05-py3 tritonserver --model-repository=/models

# Test Triton server

In [ ]:
# Install Triton client
%pip install tritonclient[http] --user --upgrade

In [28]:
# Imports
import tritonclient.http as tritonhttpclient
import gevent.ssl
import cv2
import numpy as np

In [35]:
# Get scoring URI and key
endpoint = ml_client.online_endpoints.get(endpoint_name)
scoring_uri = endpoint.scoring_uri
keys = ml_client.online_endpoints.get_keys(endpoint_name)
auth_key = keys.primary_key

# Initialize client handler
triton_client = tritonhttpclient.InferenceServerClient(
    url=url,
    ssl=True,
    ssl_context_factory=gevent.ssl._create_default_https_context,
)

# Create headers
headers = {}
headers["Authorization"] = f"Bearer {auth_key}"

In [36]:
# Check Triton server is ready
health_ctx = triton_client.is_server_ready(headers=headers)
print("Is server ready - {}".format(health_ctx))

Is server ready - True


In [37]:
# List models on Triton server
triton_client.get_model_repository_index(headers=headers)

[{'name': 'yolov8l', 'version': '1', 'state': 'READY'},
 {'name': 'yolov8n', 'version': '1', 'state': 'READY'}]

In [77]:
# Check ready model status
model_name = "yolov8l"
version = "1"
triton_client.is_model_ready(model_name, version, headers)

True

In [78]:
# Load image file
img = cv2.imread('test.jpg')

# Preprocess image
img = cv2.resize(img, (640, 640))
img = img.astype(np.float32) / 255.0
img = np.transpose(img, (2, 0, 1))
img = np.expand_dims(img, axis=0)

# Create inference request
input = tritonhttpclient.InferInput('images', img.shape, 'FP32')
input.set_data_from_numpy(img)

In [79]:
# Send inference request
outputs = [tritonhttpclient.InferRequestedOutput('output0')]
response = triton_client.infer('yolov8n', inputs=[input], outputs=outputs, headers=headers)

In [80]:
# Get rid of first dimension (we have just one image)
output_data = response.as_numpy('output0')[0]

# Transpose to get 8400x84
output_data = output_data.transpose()

# Now each row is one bounding box
# First 4 numbers are bounding box coordinates, next 80 are probabilities for different classes
print(f"Row 0: x_center: {output_data[0][0]} y_center: {output_data[0][1]} width: {output_data[0][2]} height: {output_data[0][3]}")
print(f"Classes probabilities: {output_data[0][4:]}")

Row 0: x_center: 3.311304807662964 y_center: 5.113790035247803 width: 6.834377765655518 height: 10.289277076721191
Classes probabilities: [ 2.7716e-06  2.6822e-07  9.2387e-07  1.7881e-07  8.9407e-07  3.5763e-07  2.6822e-07  4.7684e-07  9.2387e-07  7.7486e-07  1.7881e-07  4.4703e-07  1.7881e-07  3.5763e-07  9.8348e-07  1.1921e-07  5.9605e-08  2.3842e-07  1.7881e-07  2.6822e-07  1.1921e-07  2.3842e-07  5.9605e-08  2.9802e-07  1.4901e-07  7.7486e-07
  1.7881e-07  1.7881e-07  8.9407e-08  4.1723e-07  5.3644e-07  3.5763e-07  3.5763e-07  9.8348e-07  3.2783e-07  2.9802e-08  2.0862e-07  9.2387e-07  2.9802e-07  2.9802e-07  6.8545e-07  7.4506e-07  2.6822e-07  2.0862e-07  2.0862e-07  4.1723e-07  2.6822e-07  2.6822e-07  1.7881e-07  2.6822e-07  8.9407e-08  1.4901e-07
  1.7881e-07  1.4901e-07  1.7881e-07  1.1921e-07  6.8545e-07  1.4901e-07  4.1723e-07  2.0862e-07  5.3644e-07  2.6822e-07  2.0862e-07  2.0862e-07           0  1.4901e-07  2.9802e-08  2.0862e-07  1.1921e-07  8.9407e-08  8.9407e-08  3.8743

In [90]:
# Find bounding box with highest probability and print class id
yolo_classes = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", "dog", "horse",
    "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella", "handbag", "tie",
    "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove",
    "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon",
    "bowl", "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut",
    "cake", "chair", "couch", "potted plant", "bed", "dining table", "toilet", "tv", "laptop", "mouse",
    "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "book",
    "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
]

prob = output_data[0][4:].max()
class_id = output_data[0][4:].argmax()
print(f"Label: {yolo_classes[class_id]} with probability: {prob}")

Label: person with probability: 2.771615982055664e-06


In [91]:
# Cycle through all bounding boxes and print those with probability higher than 0.5
for row in output_data:
    prob = row[4:].max()
    class_id = row[4:].argmax()
    if prob > 0.5:
        print(f"Label: {yolo_classes[class_id]} with probability: {prob}")

Label: person with probability: 0.5086160898208618
Label: person with probability: 0.6555760502815247
Label: person with probability: 0.5593835711479187
Label: person with probability: 0.505821704864502
Label: person with probability: 0.5590744614601135
Label: person with probability: 0.748077929019928
Label: person with probability: 0.5721433162689209
Label: person with probability: 0.5527952909469604
Label: person with probability: 0.5461987853050232
Label: skateboard with probability: 0.5450615286827087
Label: skateboard with probability: 0.5487391352653503
Label: skateboard with probability: 0.5443091988563538
Label: person with probability: 0.8153501749038696
Label: person with probability: 0.8281033635139465
Label: person with probability: 0.8367499113082886
Label: person with probability: 0.73560631275177
Label: person with probability: 0.5858952403068542
Label: person with probability: 0.8325259685516357
Label: person with probability: 0.8535487055778503
Label: person with prob